In [1]:
from Game import Game


In [5]:
players_data = [('Alice', 1000), ('Bob', 1000), ('Charlie', 1000)]
game = Game(players_data)

game.run_hand()

Charlie posts small blind: 5
Alice posts big blind: 10
Bob's turn.
Hand: [5♥, 2♠]
Stack: 1000
Amount to call: 10
Legal actions: ['fold', 'call', 'raise']
Bob calls.
Charlie's turn.
Hand: [5♣, 6♦]
Stack: 995
Amount to call: 10
Legal actions: ['fold', 'call', 'raise']
Charlie calls.
Alice's turn.
Hand: [T♣, 8♦]
Stack: 990
Amount to call: 10
Legal actions: ['check', 'bet']
Alice checks.
Community cards: [2♥, 8♣, 2♦]
Charlie's turn.
Hand: [5♣, 6♦]
Stack: 990
Amount to call: 0
Legal actions: ['check', 'bet']
Charlie checks.
Alice's turn.
Hand: [T♣, 8♦]
Stack: 990
Amount to call: 0
Legal actions: ['check', 'bet']
Alice checks.
Bob's turn.
Hand: [5♥, 2♠]
Stack: 990
Amount to call: 0
Legal actions: ['check', 'bet']
Bob checks.
Community cards: [2♥, 8♣, 2♦, 3♣]
Charlie's turn.
Hand: [5♣, 6♦]
Stack: 990
Amount to call: 0
Legal actions: ['check', 'bet']
Charlie checks.
Alice's turn.
Hand: [T♣, 8♦]
Stack: 990
Amount to call: 0
Legal actions: ['check', 'bet']
Alice checks.
Bob's turn.
Hand: [5♥, 2♠

In [3]:
import unittest
from Game import Game

class TestGame(unittest.TestCase):
    def test_game_creation(self):
        players_data = [('Alice', 1000), ('Bob', 1000)]
        game = Game(players_data)
        self.assertEqual(len(game.players), 2)
        self.assertEqual(game.players[0].name, 'Alice')
        self.assertEqual(game.small_blind, 5)
        self.assertEqual(game.big_blind, 10)

    def test_setup_hand(self):
        players_data = [('Alice', 1000), ('Bob', 1000), ('Charlie', 1000)]
        game = Game(players_data)
        game._setup_hand()

        self.assertEqual(game.pot, 15)
        self.assertEqual(len(game.community_cards), 0)
        self.assertEqual(len(game.players[0].hand), 2)
        self.assertEqual(len(game.players[1].hand), 2)
        self.assertEqual(len(game.players[2].hand), 2)
        self.assertEqual(game.players[2].current_bet, 5)
        self.assertEqual(game.players[0].current_bet, 10)

    def test_pre_flop_betting_all_fold_to_bb(self):
        players_data = [('Alice', 1000), ('Bob', 1000), ('Charlie', 1000)]
        game = Game(players_data)
        game._setup_hand()

        # Scripted actions: Bob folds, Charlie folds
        game.players[1].scripted_actions = [('fold', 0)]
        game.players[2].scripted_actions = [('fold', 0)]

        game._run_betting_round()

        # Alice (BB) should win the pot
        self.assertEqual(game.players[0].stack, 1005) # 1000 - 10 (bb) + 15 (pot)
        self.assertEqual(game.players[1].stack, 1000)
        self.assertEqual(game.players[2].stack, 995)

    def test_post_flop_betting_check_around(self):
        players_data = [('Alice', 1000), ('Bob', 1000), ('Charlie', 1000)]
        game = Game(players_data)
        game._setup_hand()

        # Pre-flop: Bob calls, Charlie calls, Alice checks
        game.players[1].scripted_actions = [('call', 0)]
        game.players[2].scripted_actions = [('call', 0)]
        game.players[0].scripted_actions = [('check', 0)]
        game._run_betting_round()

        game._deal_community_cards(3)

        # Post-flop: all check
        game.players[1].scripted_actions = [('check', 0)]
        game.players[2].scripted_actions = [('check', 0)]
        game.players[0].scripted_actions = [('check', 0)]
        game._run_betting_round()

        self.assertEqual(game.pot, 30)

    def test_bet_raise_and_call(self):
        players_data = [('Alice', 1000), ('Bob', 1000), ('Charlie', 1000)]
        game = Game(players_data)
        game._setup_hand()

        # Pre-flop actions
        game.players[1].scripted_actions = [('call', 0)]
        game.players[2].scripted_actions = [('raise', 30)]
        game.players[0].scripted_actions = [('call', 0)]
        game.players[1].scripted_actions.append(('call', 0))

        game._run_betting_round()

        self.assertEqual(game.pot, 90)
        self.assertEqual(game.players[0].stack, 970)
        self.assertEqual(game.players[1].stack, 970)
        self.assertEqual(game.players[2].stack, 970)